In [1]:
# Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
# 체인을 수동으로 구현해야 합니다.
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
# Aaronson 은 유죄인가요?
# 그가 테이블에 어떤 메시지를 썼나요?
# Julia 는 누구인가요?
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain.document_loaders import TextLoader
text_loader = TextLoader('./documents/document.txt')

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators=[
        '\n\n','\n'
    ],
    chunk_size=600,
    chunk_overlap=100,
)
docs = text_loader.load_and_split(text_splitter=text_splitter)

In [4]:
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS

CACHE_DIR = './.cache'
embeddings = OpenAIEmbeddings()
cache_store = LocalFileStore(CACHE_DIR)
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_store)
vector_store = FAISS.from_documents(docs, cache_embeddings)

In [5]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True,
)


In [6]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
prompt = ChatPromptTemplate.from_messages([
    ('system', """
    You are good at replying users questions based on below context.\n\n

    context: {context},
    ------
    """),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{question}'),
])
llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.1,
)
questions = [
   "Is Aaronson guilty?",
   "What messages did he write on the table?",
   "Who is Julia?",
]

In [7]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
chain = {"question": RunnablePassthrough(), "context": vector_store.as_retriever(), 'chat_history': RunnableLambda(lambda _: memory.load_memory_variables({})['chat_history'])}| prompt | llm


In [8]:
def ask_something(question:str):
    print(question)
    print('----------')
    answer = chain.invoke(question)
    print(answer.content)
    print('----------\n')
    memory.save_context(
        inputs={
            'human': question,
        },
        outputs={
            'ai': answer.content,
        }
    )
for q in questions:
    ask_something(q)

Is Aaronson guilty?
----------
According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
----------

What messages did he write on the table?
----------
He wrote the following messages on the table:

1. FREEDOM IS SLAVERY
2. TWO AND TWO MAKE FIVE
3. GOD IS POWER
----------

Who is Julia?
----------
Julia is a character in the text who is associated with the protagonist. She is someone whom the protagonist loves and cares for deeply.
----------

